In [15]:
import pandas as pd
import numpy as np
import pickle
from scipy import sparse
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [16]:
df = pd.read_csv('D:/Python Projects/data/Book1.csv', encoding = 'latin1', dtype = 'object')
df.head(n=5)

,CLM_SK,Fail_Short_PartNo,Labor_Cost_USD,Part_Cost_USD,Days_To_Fail_MinZero,Miles_To_Fail,Text_Cluster_Family,SYMP_CLASS_NM,Customer_Contention_Text,PROBLEM,Target
0,76237140,81690,97.33,252.88,483,13949,NOISE/VIBRATION,SEATS (FRONT)/LEFT SIDE/NOISE/VIBRATION,CK FOR THE DRIVERS SIDE 2ND ROW SEAT WILL NOT ...,Unconfirmed,0
1,103326125,57455,436.82,192.64,1265,46722,NOISE/VIBRATION,"ABS/VSA SYSTEM/SENSOR, FR/NOISE/VIBRATION",CONTINUATION OF FIRST LINE. ADDED FOR TECHNICI...,Unconfirmed,0
2,101187649,57306,535.5,1293.16,274,17819,WARNING LIGHT ON,HYBRID SYSTEM/TANDEM MOTOR CYLINDER/WARNING LI...,C/S BRAKE SYSTEM WARNING LIGHT & OTHER WARNING...,Unconfirmed,0
3,88661240,57111,1261.5,610.51,5,205,WARNING LIGHT ON,ABS/VSA SYSTEM/MODULATOR ASSY/WARNING LIGHT ON,CLIENTS STATES THERE IS A WARNING LIGHT ON AND...,Unconfirmed,0
4,79593206,57111,1009.87,463.58,467,30655,NOT APPL,ABS/VSA SYSTEM/MODULATOR ASSY/FUNCTION ISSUE,VERIFIER PEDALE DE FREIN . DESCEND AU FOND ET ...,Unconfirmed,0


In [17]:
df.Customer_Contention_Text = df.Customer_Contention_Text.astype('str')
df.Target = df.Target.astype('int')
df.Labor_Cost_USD = df.Labor_Cost_USD.astype('float')
df.Part_Cost_USD = df.Part_Cost_USD.astype('float')
df.Days_To_Fail_MinZero = df.Days_To_Fail_MinZero.astype('float')
df.Miles_To_Fail = df.Miles_To_Fail.astype('float')

df.dtypes

CLM_SK                       object
Fail_Short_PartNo            object
Labor_Cost_USD              float64
Part_Cost_USD               float64
Days_To_Fail_MinZero        float64
Miles_To_Fail               float64
Text_Cluster_Family          object
SYMP_CLASS_NM                object
Customer_Contention_Text     object
PROBLEM                      object
Target                        int32
dtype: object

In [18]:
df['Target'].value_counts()

0    1514
1     158
Name: Target, dtype: int64

In [19]:

enc_label = preprocessing.LabelEncoder()
df['PART5'] = enc_label.fit_transform(df.Fail_Short_PartNo)
df['TEXT_CLUSTER'] = enc_label.fit_transform(df.Text_Cluster_Family)
df['SYMP_CLASS'] = enc_label.fit_transform(df.SYMP_CLASS_NM)

df.head(n=5)

,CLM_SK,Fail_Short_PartNo,Labor_Cost_USD,Part_Cost_USD,Days_To_Fail_MinZero,Miles_To_Fail,Text_Cluster_Family,SYMP_CLASS_NM,Customer_Contention_Text,PROBLEM,Target,PART5,TEXT_CLUSTER,SYMP_CLASS
0,76237140,81690,97.33,252.88,483.0,13949.0,NOISE/VIBRATION,SEATS (FRONT)/LEFT SIDE/NOISE/VIBRATION,CK FOR THE DRIVERS SIDE 2ND ROW SEAT WILL NOT ...,Unconfirmed,0,31,3,56
1,103326125,57455,436.82,192.64,1265.0,46722.0,NOISE/VIBRATION,"ABS/VSA SYSTEM/SENSOR, FR/NOISE/VIBRATION",CONTINUATION OF FIRST LINE. ADDED FOR TECHNICI...,Unconfirmed,0,30,3,6
2,101187649,57306,535.50,1293.16,274.0,17819.0,WARNING LIGHT ON,HYBRID SYSTEM/TANDEM MOTOR CYLINDER/WARNING LI...,C/S BRAKE SYSTEM WARNING LIGHT & OTHER WARNING...,Unconfirmed,0,29,6,21
3,88661240,57111,1261.50,610.51,5.0,205.0,WARNING LIGHT ON,ABS/VSA SYSTEM/MODULATOR ASSY/WARNING LIGHT ON,CLIENTS STATES THERE IS A WARNING LIGHT ON AND...,Unconfirmed,0,28,6,5
4,79593206,57111,1009.87,463.58,467.0,30655.0,NOT APPL,ABS/VSA SYSTEM/MODULATOR ASSY/FUNCTION ISSUE,VERIFIER PEDALE DE FREIN . DESCEND AU FOND ET ...,Unconfirmed,0,28,4,3


In [6]:
from sklearn.utils import resample

df_Upsmpl = resample(df.query("Target == 1"), 
                                    replace=True, 
                                    n_samples = df.query("Target == 0").shape[0], 
                                    random_state = 321)

df_Upsmpl

,CLM_SK,Fail_Short_PartNo,Labor_Cost_USD,Part_Cost_USD,Days_To_Fail_MinZero,Miles_To_Fail,Text_Cluster_Family,SYMP_CLASS_NM,Customer_Contention_Text,PROBLEM,Target,PART5,TEXT_CLUSTER,SYMP_CLASS
45,78257671,46402,124.50,16.57,176.0,4756.0,FUNCTION ISSUE,"MASTER CYLINDER/BOOSTER/TUBE, MASTER POWER/FUN...",REPAIR WHEN CAR IS STARTED COLD BRAKE PEDAL WA...,Confirmed,1,20,1,42
680,91239783,01469,272.50,1313.38,152.0,9175.0,WARNING LIGHT ON,MASTER CYLINDER/BOOSTER/POWER ASSY/WARNING LIG...,CUSTOMER STATES: CHECK BRAKE SYSTEM LIGHT IS O...,Possible,1,0,6,39
50,69736232,46402,23.00,19.54,40.0,1856.0,FUNCTION ISSUE,"MASTER CYLINDER/BOOSTER/TUBE, MASTER POWER/FUN...",HAVE TO PRESS HARD ON BRAKE PEDAL,Confirmed,1,20,1,42
60,95864099,46402,11.80,22.72,57.0,1381.0,FUNCTION ISSUE,"MASTER CYLINDER/BOOSTER/TUBE, MASTER POWER/FUN...",CUST STATES BRAKE PEDAL HARD ON FIRST APPLICAT...,Possible,1,20,1,42
701,73878168,01469,517.61,230.53,0.0,21.0,NOT APPL,MASTER CYLINDER/BOOSTER/POWER ASSY/DEFECTIVE,DURING PDI- BRAKES WILL LOCK ON WHEN DRIVING,Possible,1,0,4,31
710,71284633,01469,132.00,77.03,61.0,2064.0,FUNCTION ISSUE,MASTER CYLINDER/BOOSTER/POWER ASSY/FUNCTION ISSUE,CUSTOMER STATES THE BRAKES FEEL REALLY HARD TO...,Possible,1,0,1,32
657,94503770,01469,153.40,52.83,8.0,418.0,FUNCTION ISSUE,MASTER CYLINDER/BOOSTER/POWER ASSY/FUNCTION ISSUE,CUST STATES FIRST TIME BRAKES ARE APPLIED WHEN...,Confirmed,1,0,1,32
91,74172123,46402,33.39,18.07,154.0,7022.0,FUNCTION ISSUE,"MASTER CYLINDER/BOOSTER/TUBE, MASTER POWER/FUN...",VERIFIER LA PEDAL DE FFREIN TRES TRES DIFFICIL...,Possible,1,20,1,42
383,100346712,46101,184.00,482.70,1548.0,81411.0,NOT APPL,MASTER CYLINDER/BOOSTER/MASTER CYLINDER/FUNCTI...,"TOW IN, CUSTOMER STATES HAD BASICALLY NO BRAKE...",Confirmed,1,16,4,25
685,86645819,01469,154.70,74.51,114.0,8785.0,FUNCTION ISSUE,MASTER CYLINDER/BOOSTER/POWER ASSY/FUNCTION ISSUE,CLIENT STATES THAT BRAKE PEDAL IS HARD FIRST T...,Possible,1,0,1,32


In [7]:
df_concat = pd.concat([df.query("Target == 0"), df_Upsmpl])

df_concat["Target"].value_counts()

1    1514
0    1514
Name: Target, dtype: int64

In [8]:
df_train, df_test, Trgt_train, Trgt_test = train_test_split(df_concat.drop(['Target'], axis=1), df_concat['Target'], test_size = .1, random_state = 12)


In [9]:
## Declare encoders TFidf functions
enc_onehot = OneHotEncoder()
count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()

In [10]:
##Encode and TFidf and combine the training data

# Hot Encode the categorical variables to take away order
X_train_PNO_HotEncode = enc_onehot.fit_transform(df_train.PART5.values.reshape(-1,1))

# TFidf Complaint data
X_train_Cmplnt_Counts = count_vect.fit_transform(df_train.Customer_Contention_Text)
X_train_Cmplnt_tfidf = tfidf_transformer.fit_transform(X_train_Cmplnt_Counts)

# Combine the factor data
X_train_Combi_tfidf = sparse.hstack((X_train_PNO_HotEncode, X_train_Cmplnt_tfidf))

In [13]:
## Ecnode and TFidf and combine the test data

# Hot Encode the categorical variables to take away order
X_test_PNO_HotEncode = enc_onehot.transform(df_test.PART5.values.reshape(-1,1))

#TFidf Complaint data
X_test_Cmplnt_Counts = count_vect.transform(df_test.Customer_Contention_Text)
X_test_Cmplnt_tfidf = tfidf_transformer.transform(X_test_Cmplnt_Counts)

# Combine the factor data
X_test_Combi_tfidf = sparse.hstack((X_test_PNO_HotEncode, X_test_Cmplnt_tfidf))

In [14]:
clf = MultinomialNB().fit(X_train_Combi_tfidf, Trgt_train)


clf.score(X_test_Combi_tfidf, Trgt_test)

0.91419141914191415